## What is Ensemble Learning?
Ensemble Learning is a process using which multiple machine learning models (such as classifiers) are strategically constructed to solve a particular problem. 

Ensemble Learning is a process using which multiple machine learning models (such as classifiers) are strategically constructed to solve a particular problem. 

source:
1. https://www.datacamp.com/community/tutorials/ensemble-learning-python
2. https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler
data = pd.read_csv('input/breast-cancer-wisconsin.data',names=['Sample code number','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class' ])
data.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [2]:
data.shape

(699, 11)

In [3]:
data.drop(['Sample code number'],axis = 1, inplace = True)
data.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


It is always important the briefly look at the data structure of the dataframe so that we would could see if there is any thing abnormal in the dataset.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
Clump Thickness                699 non-null int64
Uniformity of Cell Size        699 non-null int64
Uniformity of Cell Shape       699 non-null int64
Marginal Adhesion              699 non-null int64
Single Epithelial Cell Size    699 non-null int64
Bare Nuclei                    699 non-null object
Bland Chromatin                699 non-null int64
Normal Nucleoli                699 non-null int64
Mitoses                        699 non-null int64
Class                          699 non-null int64
dtypes: int64(9), object(1)
memory usage: 54.7+ KB


It is seen that the 'Bare Nuclei' column is intepreted as '699 non-null object'. This means that there are missing values. After observing the dataset, we found that the missing values are written as '?' in the 'Bare Nuclei' column. We then replace the missing values for 0.

In [5]:
data.replace('?',0, inplace=True)

Convert datafram to np array.

In [6]:
values=data.values

As you can see from above, there are missing values, one each in the second and the third columns. We need to calculate the values for these two cells. For this example, we’ll be using the strategy ‘mean’ and imputing along the columns, as imputing along the rows makes no sense here. The code for this is as follows:

missing_values(The placeholder for the missing values. All occurrences of missing_values will be imputed. For missing values encoded as np.nan, use the string value “NaN”.) : integer or “NaN”, optional (default=”NaN”)



strategy(The imputation strategy.) : string, optional (default=”mean”)
If “mean”, then replace missing values using the mean along the axis.

axis(The axis along which to impute.) : integer, optional (default=0)
If axis=0, then impute along columns.

verbose(Controls the verbosity of the imputer.) : integer, optional (default=0)



copy : boolean, optional (default=True)

In [7]:
imputer = Imputer()
imputedData = imputer.fit_transform(values)

C:\Users\atseng17\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


### Normalization (Scaling)
Next, we apply scaling on the test dataset, and apply the same transform to the test dataset. After fitting, the scale and offset used with your training data is stored. We use that on the test dataset with scaler.transform. To address this problem, you will normalize the ranges of the features to a uniform range, in this case, 0 - 1.

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
normalizedData = scaler.fit_transform(imputedData)

### Bootstrap AGGregating - BAGGing

#### What's bootstrapping?
In sampling without replacement, each sample unit of the population has only one chance to be selected in the sample. For example, if one draws a simple random sample such that no unit occurs more than one time in the sample, the sample is drawn without replacement. If a unit can occur one or more times in the sample, then the sample is drawn with replacement.

Next, we create subsets  with n' smaples from the n samples in one bag with replacement, so we have m bags. This is called bootstrapping.

In [9]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [10]:
X = normalizedData[:,0:9]
Y = normalizedData[:,9]

Let's first see the original way of using decision trees

In [11]:
results_DT = cross_val_score(DecisionTreeClassifier(max_depth=6), X, Y, cv=10)
print(results_DT.mean())

0.9415067214883506


One can further use GridSearchCV to find the best parameter.

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': np.arange(3, 10)}
tree = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=10)
tree.fit(X, Y)
print("tuned hpyerparameters :(best parameters) ",tree.best_params_)
print("accuracy :",tree.best_score_)

tuned hpyerparameters :(best parameters)  {'max_depth': 9}
accuracy : 0.9456366237482118


We got the best score of 94.85%. Could we imporve it via Bagging?

#### What is the good thing of ensemble methods?
- lower bias
- less overfitting

#### Bagged Decision Trees
In this example, we dicided to use 100 DecisionTreeClassifiers. In other words, we are using 100 different decision tree models to train the model and get the average results. The base_estimator means that we have to choose a basemodel, if we choose svm, then we creat n numbers of svm models to train the model. In this case we choose decision trees.

In [13]:
kfold = model_selection.KFold(n_splits=10, random_state=7)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=7)
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.9585714285714285


### Adaptive Boosting - Ada Boosting
In boosting, we first choose random sample from the original data to the first subset and trained by the first model. Next, we find out the sample that are not well predicted. Those points will be given a higher chance to be put into the second subset. By now we have two models and two subsets. The sample that are not well predicted with the "previous models" will be given a higher chance to be put into the third subset. This precedure continues until the last subset.

#### The downsides of Ada Boosting?
- mroe overfitting than bagging

In [14]:
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 70
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.9557142857142857


In this case, both the bagging and boosting method have better results that the original decision tree model.